In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/Modeled Databases/final_df_complete.csv")


In [3]:
# The goal of this exercise will be, for every Constructor, to modelate 0s for repeated Drivers after every Season and 1s for different ones. Thus we will get a binary idea of how many times a Team switched Drivers between Seasons.

# Step one, pulverize the problem into various subsets of things. Let's only get the Constructor, their Season, and with which Driver they competed. 

constructors = dataset.groupby('constructor').apply(lambda a: a.drop('constructor', axis=1)[:])

df = constructors.reset_index().drop(['level_1'], axis=1)

driver_change_dataset = df.sort_values(by=['season','constructor','driver']).reset_index().drop(['index'], axis=1)

driver_change_dataset.head(10)

,constructor,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,...,driver_standings_pos_after_race.1,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age,wins_percentage
0,alfa,1983,1,jacarepagua,False,False,True,False,False,baldi,...,0,0,0,0,0,0,0,1.454,29,NaN
1,alfa,1983,2,long_beach,True,False,False,False,False,baldi,...,19,0,0,0,0,0,14,3.953,29,NaN
2,alfa,1983,3,ricard,False,False,False,False,True,baldi,...,26,0,0,19,0,0,14,2.946,29,NaN
3,alfa,1983,4,imola,False,False,False,False,False,baldi,...,28,0,0,26,0,0,12,3.762,29,0.0
4,alfa,1983,5,monaco,False,False,False,True,False,baldi,...,19,0,0,28,0,0,11,3.799,29,0.0
5,alfa,1983,6,spa,False,False,False,False,False,baldi,...,12,0,0,19,1,0,8,4.610,29,NaN
6,alfa,1983,7,detroit,True,False,False,False,False,baldi,...,13,1,0,12,1,0,9,5.182,29,0.0
7,alfa,1983,8,villeneuve,True,False,False,False,False,baldi,...,14,1,0,13,1,0,10,6.026,29,0.0
8,alfa,1983,9,silverstone,False,False,False,False,False,baldi,...,13,1,0,14,1,0,9,3.398,29,NaN
9,alfa,1983,10,hockenheimring,False,False,True,False,False,baldi,...,14,1,0,13,1,0,9,2.539,29,NaN


In [4]:
# Now here's the trick, we will use a where clause to codify our data.

driver_change_dataset['change_drivers'] = np.where((driver_change_dataset['constructor'] == driver_change_dataset['constructor'].shift(periods=-1,axis=0)) & (driver_change_dataset['driver'] == driver_change_dataset['driver'].shift(periods=-1,axis=0)),0,1)


In [5]:
# Let's check Mclaren's career again, we should see the correct codification now!

driver_change_dataset[['constructor','season','driver','change_drivers']][(driver_change_dataset['constructor']=='mclaren')].head(50)

,constructor,season,driver,change_drivers
161,mclaren,1983,lauda,0
162,mclaren,1983,lauda,0
163,mclaren,1983,lauda,0
164,mclaren,1983,lauda,0
165,mclaren,1983,lauda,0
166,mclaren,1983,lauda,0
167,mclaren,1983,lauda,0
168,mclaren,1983,lauda,0
169,mclaren,1983,lauda,0
170,mclaren,1983,lauda,0


In [6]:
# Now let's sort things out in order to be able to use the dataset as a time series.

driver_change_dataset = driver_change_dataset.sort_values(by=['season','round','constructor','driver']).reset_index().drop(['index'], axis=1)


In [7]:
driver_change_dataset.to_csv(r'/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/driver_change_dataset.csv', index = False)